    Retrieve maximum BRDF first.

In [4]:
%run MCM_SMB_retrieval.py

ERROR:root:File `'MCM_SMB_retrieval.py'` not found.



    Author@Yizhe

    Created on Oct 1, 2019

    Retrieve the surface ID from the maxBRF_XX_DoY.nc dataset.  
    Running this script to generate SurfaceID_PTA_xxx.nc data files storing the surface ID and the representative BRDF for each surface category for the specified PTA.

In [5]:
import numpy as np
import xarray as xr
from sklearn.cluster import MiniBatchKMeans

ModuleNotFoundError: No module named 'sklearn'

    use xarray.open_mfdataset to implement dask multi-processor.  

    the data array -- target_max_brf -- contains three fillin values:  
    
    (1) -997. : these points are preset for water category (consist of shallow and deep ocean);  
    (2) -998. : these points are preset for coastal category (consist of coastline, P. shallow inland water, S. inland water, P. deep inland water);  
    (3) -999. : these points are neither MAIA-water nor MAIA-coastal and have invalid RTLS coefficients for more than 5 years.

In [ ]:
DS = xr.open_mfdataset('MAIAC_SMB/maxBRF_LA_048.nc')
print(DS)

    ===========K-means clustering===========

In [ ]:
sel_data = DS.max_BRF.sel(vza=[2.5, 27.5, 42.5, 62.5, 72.5], 
                          raz=[22.5, 52.5, 97.5, 127.5, 157.5]
                         )

# sel_data = DS.max_BRF
filled_sel_data = sel_data.fillna(-996.)
print(filled_sel_data)

In [ ]:
sample_data = filled_sel_data[:, :, 0, 0, 0]
idx_coastal = np.where(sample_data==-997.)
idx_water = np.where(sample_data==-998.)
idx_unknown = np.where(sample_data==-996.)

In [ ]:
# print(idx_coastal[0].shape[0], idx_water[0].shape[0], idx_unknown[0].shape[0])

In [ ]:
stacked_filled_sel_data = sel_data.stack(i=('y', 'x'), z=('cos_sza', 'vza', 'raz'))
input_data = stacked_filled_sel_data.values

In [ ]:
# whiten data -- equivalent to using sklearn.preprocessing.StandardScaler
np.place(input_data, input_data<-900., np.nan)
input_data_mean = np.nanmean(input_data, axis=0)
input_data_std = np.nanstd(input_data, axis=0)
whitened_data = (input_data - input_data_mean) / input_data_std
idx_nan = xr.ufuncs.isnan(whitened_data)
whitened_data[idx_nan] = -9999.

In [ ]:
n_clusters = 23

# cluster the whitened data
minibatch_kmeans = MiniBatchKMeans(
    n_clusters=n_clusters,
    init='k-means++',
    random_state=0,
    batch_size=1000,
    verbose=0,
    max_no_improvement=100).fit(whitened_data)

unsorted_labels = minibatch_kmeans.labels_.reshape(1000, 1000)

    ===========sort labels===========

calculate grid_mean_BRF

In [ ]:
grid_mean_BRF = stacked_filled_sel_data.unstack('i').mean(axis=0).values

use grid_mean_BRF to get unsorted_cluster_mean_BRF

In [ ]:

unsorted_cluster_mean_BRF = []
for icluster in np.arange(n_clusters):
    _cluster_BRF = grid_mean_BRF[unsorted_labels == icluster]
    unsorted_cluster_mean_BRF.append(np.nanmean(_cluster_BRF))

get sorted_labels based on unsorted_cluster_mean_BRF (small to large)

In [ ]:

# assign a number to each cluster_mean_BRF
unsorted_clusters = np.dstack([range(n_clusters), unsorted_cluster_mean_BRF])[0]

# sort the rest
sorted_clusters = np.array(sorted(unsorted_clusters, key=lambda x: x[1]))[:, 0]

# sort labels (0--n_clusters where category 0 is for unconsidered points)
sorted_labels = unsorted_labels.copy()
for i, j in enumerate(sorted_clusters):
    np.place(sorted_labels, unsorted_labels==j, 101+i)
sorted_labels -= 100

# 
sorted_labels[idx_water] = 0
sorted_labels[idx_coastal] = 1
sorted_labels[idx_unknown] = 1

use sorted_labels to calculate some statistics within each sorted cluster

In [ ]:

sorted_cluster_std_BRF = []
sorted_cluster_mean_BRF = []
sorted_cluster_tot_num = []

# plus 1 because adding water type (0)
for icluster in np.arange(n_clusters+1):
    _cluster_BRF = grid_mean_BRF[sorted_labels == icluster]
    
    sorted_cluster_mean_BRF.append(np.nanmean(_cluster_BRF))
    sorted_cluster_std_BRF.append(np.nanstd(_cluster_BRF))
    sorted_cluster_tot_num.append(len(_cluster_BRF))
    
    print "{:2d} {:6d} {:.2f} {:.2f}".format(icluster, sorted_cluster_tot_num[icluster], 
                    sorted_cluster_mean_BRF[icluster], sorted_cluster_std_BRF[icluster])
    
    

    Retrieve maximum BRF

In [ ]:
# calculate category-wise BRDF stats (max&min&mean)
ncats = np.arange(2, n_clusters)
cat_BRDF_stats = []
for icat in ncats:
    idx_cat = np.where(sorted_labels == icat)
    cat_BRDF_stats.append(np.max(TARGET_max_brf[idx_cat], axis=0))

    Save surface ID!

In [ ]:
out_nc = "SurfaceID_LA_048.nc"

MAIA_lats = DS.Latitude
MAIA_lons = DS.Longitude
MAIA_lats.to_netcdf(out_nc, 'w')
MAIA_lons.to_netcdf(out_nc, 'a')

MAIA_labels = xr.DataArray(sorted_labels, name='surface_ID',
                           dims=['y', 'x'],
                           coords = {'lat': (('y', 'x'), MAIA_lats),
                                     'lon': (('y', 'x'), MAIA_lons)})
MAIA_labels.to_netcdf(out_nc, 'a')

    ===========show labels===========

In [ ]:
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
%matplotlib notebook


DS_sfcID = xr.open_dataset('SurfaceID_LA_048.nc')
sfcID = DS_sfcID.surface_ID

ax = plt.axes(projection=ccrs.Orthographic(-100, 35))

# sfcID.plot.imshow(levels=23, vmax=22, origin='upper', cmap='RdYlBu_r')
sfcID.plot.pcolormesh('lon', 'lat', ax=ax, transform=ccrs.PlateCarree(), levels=n_clusters+2, vmax=n_clusters, cmap='RdYlBu_r');

ax.set_global()
ax.coastlines(resolution='10m')

plt.show()